In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=6, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr

In [5]:
import motor_jacobian

In [6]:
B = vsr.Biv(-1.0, 0.0, 0.0).unit() * pi/6.
R = B.exp()
print(R)

Rot: [ 0.87 0.5 0 0 ]


In [7]:
f, J_f = motor_jacobian.diff_rotor_cost(vsr.Rot(1.0,2.0,3.0,4.0), 
                                        vsr.Vec(0.0,1.0,0.0), 
                                        vsr.Vec(1.0,1.0,3.0))
f = vsr.Vec(*f)
print(f)
print(J_f)

Vec: [ -21 -11 -23 ]
[[ 4.  2. -8. -6.]
 [ 2. -4.  6. -8.]
 [-8. -6. -4. -2.]]


In [8]:
J_f2 = motor_jacobian.analytic_diff_rotor_cost(vsr.Rot(1.0,2.0,3.0,4.0), 
                                        vsr.Vec(0.0,1.0,0.0), 
                                        vsr.Vec(1.0,1.0,3.0))
print(J_f2 * -1.)

[[-0. -0. -8.  6.]
 [-0. -0. -0. -0.]
 [-0. -6.  4. -0.]]


In [9]:
B = vsr.Biv(1.0, 1.0, 1.0).unit() * pi/6.
R = B.exp()
a = vsr.Vec(1.0, 1.0, 1.0)
b = vsr.Vec(0.0, 1.0, 3)
print(R)
f, J_f = motor_jacobian.diff_rotor_cost(R, a, b)
print(J_f)

Rot: [ 0.87 -0.29 -0.29 -0.29 ]
[[ 0.57735   1.732051  2.886751 -0.57735 ]
 [ 1.732051 -0.57735   0.57735   2.886751]
 [ 2.886751 -0.57735  -0.57735  -1.732051]]


In [10]:
J_f3 = 2.* np.array([np.array(  (vsr.Vec(1,0,0) * a) * R.rev()),
                      np.array( (vsr.Vec(0,1,0) * a) * R.rev()),
                       np.array( (vsr.Vec(0,0,1) * a) * R.rev())]).copy()
print(J_f3)

[[ 0.57735   1.732051  2.886751  0.57735 ]
 [ 1.732051 -0.57735  -0.57735   2.886751]
 [ 2.886751  0.57735  -0.57735  -1.732051]]


In [11]:
(vsr.Vec(0,1,0) * a)

Rot: [ 1 -1 0 1 ]

In [12]:
(vsr.Vec(1,0,0) * a)

Rot: [ 1 1 1 0 ]

In [13]:
(vsr.Vec(1,0,0) * a) * R.rev() * 2.

Rot: [ 0.58 1.7 2.9 0.58 ]

In [14]:
np.allclose(J_f, J_f3)

False

In [15]:
J_f - J_f3

array([[ 0.      , -0.      ,  0.      , -1.154701],
       [-0.      ,  0.      ,  1.154701,  0.      ],
       [ 0.      , -1.154701,  0.      ,  0.      ]])

In [16]:
J_f3

array([[ 0.57735 ,  1.732051,  2.886751,  0.57735 ],
       [ 1.732051, -0.57735 , -0.57735 ,  2.886751],
       [ 2.886751,  0.57735 , -0.57735 , -1.732051]])

In [17]:
(vsr.Vec(1,0,0) * a) * vsr.Rot(1.0,2.0,3.0,4.0).rev()

Rot: [ 6 3 -6 -3 ]

In [18]:
print(R)
def J_x(x):
    return np.array([[-x[1], -x[2], -x[3]],
                     [x[0], -x[3],  x[2]],
                     [ x[3],   x[0], -x[1]],
                     [-x[2],  x[1],   x[0]]])
print(J_x(R))

Rot: [ 0.87 -0.29 -0.29 -0.29 ]
[[ 0.288675  0.288675  0.288675]
 [ 0.866025  0.288675 -0.288675]
 [-0.288675  0.866025  0.288675]
 [ 0.288675 -0.288675  0.866025]]


In [19]:
J_l = motor_jacobian.rotor_local_parameterization(R)
print(J_l)

[[-0.288675 -0.288675 -0.288675]
 [-0.866025 -0.288675  0.288675]
 [ 0.288675 -0.866025 -0.288675]
 [-0.288675  0.288675 -0.866025]]


In [20]:
np.array([np.array(vsr.Rot(1,0,0,0) * R),
          np.array(vsr.Rot(0.0,1.0,0.0,0.0) * R),
          np.array(vsr.Rot(0.0,0.0,1.0,0.0) * R),
          np.array(vsr.Rot(0.0,0.0,0.0,1.0) * R)
         ])

array([[ 0.866025, -0.288675, -0.288675, -0.288675],
       [ 0.288675,  0.866025, -0.288675,  0.288675],
       [ 0.288675,  0.288675,  0.866025, -0.288675],
       [ 0.288675, -0.288675,  0.288675,  0.866025]])

In [21]:
np.array([np.array(vsr.Rot(0.0,1.0,0.0,0.0) * R),
          np.array(vsr.Rot(0.0,0.0,1.0,0.0) * R),
          np.array(vsr.Rot(0.0,0.0,0.0,1.0) * R),
         ])
print(R.log())
print(R * vsr.valkenburg2(vsr.Biv(-2,0,0), R.log()*2.))
print(R * vsr.valkenburg2(vsr.Biv(0,-2,0), R.log()*2.))
print(R * vsr.valkenburg2(vsr.Biv(0,0,-2), R.log()*2.))

Biv: [ -0.3023 -0.3023 -0.3023 ]
Rot: [ -0.58 -2.2 0.38 0.1 ]
Rot: [ -0.58 0.1 -2.2 0.38 ]
Rot: [ -0.58 0.38 0.1 -2.2 ]


In [22]:
(B * B).inv()

Rot: [ -3.6 -0 -0 -0 ]

In [23]:
vsr.Biv(1,0,0) * R.log()

Rot: [ 0.3 0 -0.3 0.3 ]

In [24]:
print(J_f3)
print(J_x(R))
np.dot(-J_f3.copy(), J_x(R).copy())

[[ 0.57735   1.732051  2.886751  0.57735 ]
 [ 1.732051 -0.57735  -0.57735   2.886751]
 [ 2.886751  0.57735  -0.57735  -1.732051]]
[[ 0.288675  0.288675  0.288675]
 [ 0.866025  0.288675 -0.288675]
 [-0.288675  0.866025  0.288675]
 [ 0.288675 -0.288675  0.866025]]


array([[-1., -3., -1.],
       [-1.,  1., -3.],
       [-1., -1.,  1.]])

In [25]:
np.dot(J_f, J_l)

array([[-0.666667, -3.333333, -0.      ],
       [-0.666667,  0.      , -3.333333],
       [-0.      , -0.666667,  0.666667]])

In [26]:
a = vsr.Vec(1.,2.,3.)
print(a)
B = vsr.Biv(1.0, 2.0, 3.0).unit() * pi/6.
print(B)
R = B.exp()
print(R)
D = vsr.Biv(1.0,0.0,0.0)
D * B * B.exp()

Vec: [ 1 2 3 ]
Biv: [ 0.1399 0.2799 0.4198 ]
Rot: [ 0.87 -0.13 -0.27 -0.4 ]


Rot: [ -0.12 0.26 0.36 -0.24 ]

In [27]:
np.array([vsr.diff_rotor(R,vsr.Biv(1.0,0.0,0.0),vsr.Vec(1.0,0.0,0.0)),
          vsr.diff_rotor(R,vsr.Biv(1.0,0.0,0.0),vsr.Vec(0.0,1.0,0.0)),
          vsr.diff_rotor(R,vsr.Biv(1.0,0.0,0.0),vsr.Vec(0.0,0.0,1.0)),
          vsr.diff_rotor(R,vsr.Biv(0.0,1.0,0.0),vsr.Vec(1.0,0.0,0.0)),
          vsr.diff_rotor(R,vsr.Biv(0.0,1.0,0.0),vsr.Vec(0.0,1.0,0.0)),
          vsr.diff_rotor(R,vsr.Biv(0.0,1.0,0.0),vsr.Vec(0.0,0.0,1.0)),
          vsr.diff_rotor(R,vsr.Biv(0.0,0.0,1.0),vsr.Vec(1.0,0.0,0.0)),
          vsr.diff_rotor(R,vsr.Biv(0.0,0.0,1.0),vsr.Vec(0.0,1.0,0.0)),
          vsr.diff_rotor(R,vsr.Biv(0.0,0.0,1.0),vsr.Vec(0.0,0.0,1.0))]).reshape(3,3)

array([[-0.124367, -0.323893, -0.274685],
       [ 0.237105,  0.145128, -0.421061],
       [ 0.419813, -0.279875,  0.139938]])

In [28]:
print(R)
vsr.Biv(1.0,0.0,0.0) * vsr.Rot(0, 12, 13, 23)

Rot: [ 0.87 -0.13 -0.27 -0.4 ]


Rot: [ -12 0 23 -13 ]

In [29]:
j = np.zeros((4,3))
for i in range(3):
    biv = vsr.Biv(0.0, 0.0, 0.0)
    biv[i] = 1.0 
    j[:,i] = biv * vsr.Rot(1, 12, 13, 23)
print(j)

[[-12. -13. -23.]
 [  1. -23.  13.]
 [ 23.   1. -12.]
 [-13.  12.   1.]]


In [30]:
vsr.Biv(1,0,0) * R

Rot: [ 0.13 0.87 -0.4 0.27 ]

In [31]:
R.log().exp()

Rot: [ 0.87 0.13 0.27 0.4 ]

In [32]:
vsr.Rot(1,0,0,0)* R.log() * R * 2.

Rot: [ -0.52 -0.24 -0.48 -0.73 ]

In [33]:
 def jacrot(a, rot):
    j = np.zeros((3,3))
    for i in range(3):
        biv = vsr.Biv(0.0, 0.0, 0.0)
        biv[i] = 1.0
        j[:,i] = -np.array(biv.comm(a.spin(rot)))
    return j

In [34]:
B = vsr.Biv(1.0, 0.0, 0.0).unit() * pi/6.
R = B.exp()
a = vsr.Vec(0.0, 1.0, 0.0)
b = vsr.Vec(1.0, 1.0, 1.0)

In [35]:
2. * jacrot(a, R)

array([[-1.      , -0.      , -0.      ],
       [-1.732051, -0.      , -0.      ],
       [-0.      , -1.732051,  1.      ]])

# Compute Jacobian

In [32]:
mot = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(mot.log())

Dll: [ 0 -0.5236 0 -0.7152 -0.5 -0.1917 ]


In [33]:
p = vsr.Vec(0,0,0).null()
p.comm(vsr.Lin(vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,0)).dual())

Pnt: [ 0 0 1 0 0 ]

In [28]:
mot = vsr.Mot(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0)

In [35]:
dll = vsr.Dll(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mot0 = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [36]:
mot.comm(dll)

Mot: [ 0 0 -3 2 -5 4 0 0 ]

In [37]:
mot.acomm(dll)

Mot: [ -1 0 0 0 0 0 -7 6 ]

In [38]:
mot.acomm(mot0)

Mot: [ 0 1 2 3 4 5 6 7 ]

In [39]:
mot.acomm(dll) - mot.comm(dll) 

Mot: [ -1 0 3 -2 5 -4 -7 6 ]

In [29]:
j = np.zeros((8,6))
for i in range(6):
    dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    dll[i] = 1.0
#     j[:,i] = np.array(mot.acomm(dll)) - np.array(mot.comm(dll)) 
    j[:,i] = dll * mot 

print(j)

[[-2. -3. -4.  0.  0.  0.]
 [ 1. -4.  3.  0.  0.  0.]
 [ 4.  1. -2.  0.  0.  0.]
 [-3.  2.  1.  0.  0.  0.]
 [ 6.  7. -8.  1. -2. -3.]
 [-5.  8.  7.  2.  1. -4.]
 [-8. -5. -6.  3.  4.  1.]
 [ 7. -6.  5.  4. -3.  2.]]


In [41]:
a = vsr.Dll(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
b = mot.log()
print(a)
print(b)
mot * vsr.valkenburg(a, mot.log())

Dll: [ 0 0 1 0 0 0 ]
Dll: [ 1.571 3.142 4.712 71.27 121.1 126.4 ]


AttributeError: 'module' object has no attribute 'valkenburg'

In [42]:
b * a

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (Dll, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >) -> vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >
    2. (Dll, float) -> Dll


In [43]:
a + mot * 

SyntaxError: invalid syntax (<ipython-input-43-94d5169e331e>, line 1)

In [44]:
np.array(b * a)

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (Dll, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >) -> vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >
    2. (Dll, float) -> Dll


In [45]:
vsr.Dll(*np.array(b * a)[1:7])

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (Dll, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >) -> vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >
    2. (Dll, float) -> Dll


In [46]:
vsr.Mot(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0) + ((-b * 2.)
                                                   + vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
                                                   + (b.exp() * -1.) * (b_inv * b_inv)) * vsr.Dll(*np.array(b * a)[1:7])

TypeError: unsupported operand type(s) for +: 'Dll' and 'Mot'

In [47]:
j = np.zeros((8,6))
for i in range(6):
    dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    dll[i] = 1.0
#     print(dll)
    j[:,i] = np.array(dll.comm(mot)) + np.array(dll.acomm(mot)) 
print(j)

[[-1. -2. -3.  0.  0.  0.]
 [ 0. -3.  2.  0.  0.  0.]
 [ 3.  0. -1.  0.  0.  0.]
 [-2.  1.  0.  0.  0.  0.]
 [ 5.  6. -7.  0. -1. -2.]
 [-4.  7.  6.  1.  0. -3.]
 [-7. -4. -5.  2.  3.  0.]
 [ 6. -5.  4.  3. -2.  1.]]


In [48]:
dll = vsr.Dll(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)
dll * mot

Mot: [ -1 0 3 -2 5 -4 -7 6 ]

In [30]:
dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
jac_exp = motor_jacobian.jacobian_exp(mot, dll)
print(jac_exp)
print(jac_exp.shape)

[[-2. -3. -4.  0.  0.  0.]
 [ 1. -4.  3.  0.  0.  0.]
 [ 4.  1. -2.  0.  0.  0.]
 [-3.  2.  1.  0.  0.  0.]
 [ 6.  7. -8.  1. -2. -3.]
 [-5.  8.  7.  2.  1. -4.]
 [-8. -5. -6.  3.  4.  1.]
 [ 7. -6.  5.  4. -3.  2.]]
(8, 6)


In [50]:
np.allclose(j, jac_exp)

True

In [51]:
jac_polar = motor_jacobian.jacobian_polar(mot, mot0)
print(jac_polar)
print(jac_polar.shape)

[[ 0.240986 -0.017213 -0.034427 -0.05164   0.        0.        0.        0.      ]
 [-0.017213  0.240986 -0.034427 -0.05164   0.        0.        0.        0.      ]
 [-0.034427 -0.034427  0.189346 -0.10328   0.        0.        0.        0.      ]
 [-0.05164  -0.05164  -0.10328   0.10328   0.        0.        0.        0.      ]
 [ 0.302953 -0.368364  0.141149 -0.399348  0.10328   0.10328  -0.05164
   0.05164 ]
 [-0.333937  0.113608 -0.340823 -0.141149  0.10328   0.189346  0.034427
  -0.034427]
 [ 0.020656 -0.22033  -0.113608 -0.368364 -0.05164   0.034427  0.240986
   0.017213]
 [-0.227215 -0.020656 -0.333937 -0.302953  0.05164  -0.034427  0.017213
   0.240986]]
(8, 8)


In [52]:
dll.exp()

Mot: [ 1 0 0 0 0 0 0 0 ]

In [53]:
mot = vsr.Lin(vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,0)).dual().exp()
print(mot)
p = vsr.Vec(0,0,0).null()

Mot: [ 0.54 0 0 -0.84 0 0 0.84 0 ]


In [54]:
print(motor_jacobian.diff_point(mot,p))

[[  0.000000e+000   0.000000e+000   0.000000e+000   0.000000e+000
   -1.080605e+000   0.000000e+000   0.000000e+000   0.000000e+000]
 [  0.000000e+000   0.000000e+000   0.000000e+000   0.000000e+000
    0.000000e+000   0.000000e+000   0.000000e+000   1.080605e+000]
 [  0.000000e+000   0.000000e+000  -1.682942e+000   0.000000e+000
    0.000000e+000   0.000000e+000   0.000000e+000   0.000000e+000]
 [  0.000000e+000   1.117631e+261   1.163184e-028   3.970623e+246
    1.163184e-028   3.567276e-057   2.341787e-057   4.997218e+174]
 [  3.567276e-057   2.341787e-057   4.997218e+174   1.847067e-076
    4.506057e-144   3.380401e-067   5.415432e-067   2.171452e-028]]


In [55]:
p = vsr.Vec(0,0,0).null()
p.comm(vsr.Lin(vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,0)).dual())

Pnt: [ 0 0 1 0 0 ]

In [56]:
p.comm(mot)

Pnt: [ 0 0 0.8415 0 0 ]

In [57]:
p.acomm(mot)

Pnt: [ 0 0 0 0.5403 0 ]

In [58]:
a = vsr.Vec(1,0,0).null()
b = a.spin(vsr.Rot(vsr.Biv(1,0,0) * pi/4))
print(a)
print(b)
m = vsr.Vec(1,2,3).trs() * vsr.Rot(vsr.Biv(1,2,3).unit() * pi/6)
print(m)
c = a.spin(m)
print(c)
points_a = [a]
points_b = [b]

Pnt: [ 1 0 0 1 0.5 ]
Pnt: [ 2.22e-16 1 0 1 0.5 ]
Mot: [ 0.87 -0.13 -0.27 -0.4 -0.97 -1.4 -0.76 0.13 ]
Dls: [ 1.821 2.017 3.57 1 10.07 ]


In [59]:
c.vec() - b.vec()

Vec: [ 1.8 1 3.6 ]

In [60]:
# m = vsr.Mot(1,0,0,0,0,0,0,0)
dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
m = vsr.Dll(*np.random.rand(6)).exp()
jac_exp = motor_jacobian.jacobian_exp(m, dll)
print(jac_exp)
print(a)
print(b)
print(m)
jac_s = motor_jacobian.diff_cost(m, a, b)
print(jac_s)
np.dot(jac_s, jac_exp)

[[-0.017907 -0.073845 -0.839277  0.        0.        0.      ]
 [ 0.538368 -0.839277  0.073845  0.        0.        0.      ]
 [ 0.839277  0.538368 -0.017907  0.        0.        0.      ]
 [-0.073845  0.017907  0.538368  0.        0.        0.      ]
 [ 0.245153  0.039994 -0.766656  0.538368 -0.017907 -0.073845]
 [-0.512502  0.766656  0.039994  0.017907  0.538368 -0.839277]
 [-0.766656 -0.512502 -0.245153  0.073845  0.839277  0.538368]
 [ 0.039994 -0.245153  0.512502  0.839277 -0.073845  0.017907]]
Pnt: [ 1 0 0 1 0.5 ]
Pnt: [ 2.22e-16 1 0 1 0.5 ]
Mot: [ 0.54 0.018 0.074 0.84 0.51 0.25 0.04 0.77 ]
[[ 0.051734 -0.526119 -0.227678  0.145241 -1.076737 -0.035813 -0.14769
  -1.678554]
 [-0.526119 -0.051734 -0.145241 -0.227678  0.035813 -1.076737 -1.678554
   0.14769 ]
 [-0.227678  0.145241 -0.051734  0.526119  0.14769   1.678554 -1.076737
  -0.035813]]


array([[-0.685497,  0.734441,  0.      , -2.      ,  0.      ,  0.      ],
       [ 1.729873,  0.      ,  0.734441,  0.      , -2.      , -0.      ],
       [-0.      ,  1.729873,  0.685497,  0.      ,  0.      , -2.      ]])

In [61]:
vsr.Dll(0,0,1,0,0,0).comm(a.spin(m))

Pnt: [ 0 0.3672 0.3427 0 0 ]

In [62]:
def jac2(arg):
    j = np.zeros((5,6))
    for i in range(6):
        dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        dll[i] = 1.0
        j[:,i] = 2. * np.array(dll.comm(a.spin(arg)))
    return j

In [63]:
J = jac2(m)
tmp = J[:,0].copy()
J[:,0] = J[:,2]
J[:,2] = tmp
J[:,1] *= -1.
print(J)

[[ 0.       -0.734441 -0.685497 -2.        0.        0.      ]
 [ 0.734441 -0.        1.729873  0.       -2.        0.      ]
 [ 0.685497 -1.729873  0.        0.        0.       -2.      ]
 [ 0.       -0.        0.        0.        0.        0.      ]
 [ 0.       -0.        0.        1.729873  0.685497 -0.734441]]


In [210]:
np.array(a.spin(m)) * 2.

array([-2.469941, -4.209097,  1.748808,  2.      ,  6.718858])

In [213]:
def skew(v):
    if len(v) == 4: v = v[:3]/v[3]
    skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
    return skv - skv.T

In [215]:
skew(np.array(a.spin(m).vec()) * 2.)

array([[ 0.      , -1.748808, -4.209097],
       [ 1.748808,  0.      ,  2.469941],
       [ 4.209097, -2.469941,  0.      ]])

In [31]:
dll = vsr.Dll(0,0,0,0,0,1)
a.spin(m).comm(dll)

Pnt: [ 0 0 1 0 3.57 ]

In [32]:
import scipy.sparse as sparse

In [33]:
spjacs = [np.loadtxt('/Users/lars/devel/game/dump/ceres_solver_iteration_00{}_A.txt'.format(i)) for i in range(4)]

In [34]:
D = np.loadtxt('/Users/lars/devel/game/dump/ceres_solver_iteration_000_D.txt')
D

array([ 0.008843,  0.008881,  0.00866 ,  0.008635,  0.008635,  0.008635])

In [35]:
-2. / 0.008635

-231.61551823972206

In [36]:
0.27 / 2.

0.135

In [37]:
np.loadtxt('/Users/lars/devel/game/dump/ceres_solver_iteration_000_b.txt')

array([-1.864489, -1.      , -1.159498, -1.82805 , -1.      , -0.748313,
       -1.614547, -1.      , -0.725564, -0.931623, -1.      , -1.708073,
        0.68862 , -1.      , -1.443068, -2.281495, -1.      ,  0.820182,
       -2.657144, -1.      , -0.945362, -0.676071, -1.      , -1.690215,
        0.202711, -1.      , -1.39347 , -1.567853, -1.      , -0.988182])

In [38]:
[np.loadtxt('/Users/lars/devel/game/dump/ceres_solver_iteration_00{}_x.txt'.format(i)) for i in range(4)]

[array([-0.346302, -3.851907,  0.016381, -3.897368, -3.703013, -4.059512]),
 array([ 0.267082, -0.825742, -0.141204, -2.716673, -0.142328,  1.875994]),
 array([ 0.012805,  0.001633, -0.018875,  0.120835, -0.073943,  0.220828]),
 array([ 0.000004,  0.00002 , -0.      ,  0.000049,  0.000673,  0.000122])]

In [39]:
j = 3
x, y, d = spjacs[j][:,0],  spjacs[j][:,1],  spjacs[j][:,2]
npsparse = sparse.csc_matrix((d,(x,y)))
J = npsparse.todense()
JTJ = np.dot(J.T, J)
print(JTJ)
D2 = np.diag(JTJ)
print(D2)

[[ 1.593533  0.527877 -0.525869 -0.627617  0.710904  0.      ]
 [ 0.527877  1.695614  0.697878 -0.476785  0.        0.687601]
 [-0.525869  0.697878  2.377506  0.       -0.570645  0.726547]
 [-0.627617 -0.476785  0.        0.745585  0.        0.      ]
 [ 0.710904  0.       -0.570645  0.        0.745585  0.      ]
 [ 0.        0.687601  0.726547  0.        0.        0.745585]]
[ 1.593533  1.695614  2.377506  0.745585  0.745585  0.745585]


In [40]:
D

array([ 0.008843,  0.008881,  0.00866 ,  0.008635,  0.008635,  0.008635])

In [41]:
D / D2

array([ 0.005549,  0.005238,  0.003642,  0.011582,  0.011582,  0.011582])

In [42]:
np.array(c.vec() - b.vec())

array([ 1.821429,  1.017169,  3.570053])

In [43]:
import scipy.optimize as opt

In [131]:
motor = vsr.Vec(1,2,3).trs() * vsr.Rot(vsr.Biv(1,2,3).unit() * np.pi/6.0)
print(motor)

Mot: [ 0.87 -0.13 -0.27 -0.4 -0.97 -1.4 -0.76 0.13 ]


In [147]:
n_points = 100
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.9, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [148]:
def func(arr):
    return np.array([np.array(a.spin(vsr.Dll(*arr).exp()).vec() - b.vec()) 
                     for a, b in zip(points_a, points_b)]).ravel()

In [149]:
init_dll = np.array([0,0,0,0,0,0]).reshape(6,1)
current_mot = np.array([1,0,0,0,0,0,0,0]).reshape(8,1)

In [150]:
its = 0

In [151]:
def jac(arg):
    global its
    global current_mot
    current_mot = np.array(vsr.Dll(*arg).exp() * vsr.Mot(*current_mot))
    dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    
    jac_exp = motor_jacobian.jacobian_exp(vsr.Mot(*current_mot), dll)
#     print(jac_exp)

    js_ = np.array([motor_jacobian.diff_cost(vsr.Mot(*current_mot), a, b)
                   for a, b in zip(points_a, points_b)])
    
#     print(js_)
    
    js = np.array([np.dot(motor_jacobian.diff_cost(vsr.Mot(*current_mot), a, b), jac_exp)
                   for a, b in zip(points_a, points_b)])
    
#     print(js)
    
    js = js.reshape(-1,6)
    return js

In [152]:
def __jac(m,a,b):  
    j = np.zeros((3,6))
    for i in range(6):
        dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        dll[i] = 1.0
        j[:,i] = 2. * np.array(dll.comm(a.spin(vsr.Mot(*m))).vec()).copy()
    return j

def jac3(arg):
    global current_mot
    current_mot = np.array(vsr.Dll(*arg).exp() * vsr.Mot(*current_mot)).copy()
    
    js_ = np.array([__jac(current_mot,a,b) for a, b in zip(points_a, points_b)]).reshape(-1,6)
    
    return js_

In [153]:
# dll_ = np.random.rand(6)
# print(jac3(dll_))

In [154]:
# print(jac3(dll_))

In [155]:
# print(func(init_dll))
# jac(init_dll)

In [156]:
# dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# dll[1] = 1.0
# 2. * np.array(dll.comm(a.spin(m)).vec())

In [157]:
out = opt.leastsq(func, init_dll, Dfun=jac3, full_output=False)
print(out)

(array([-0.139935, -0.279875, -0.419813, -1.033147, -1.453353, -0.807249]), 2)


In [158]:
estimated_motor = vsr.Dll(*out[0]).exp()
print(estimated_motor)

Mot: [ 0.87 -0.13 -0.27 -0.4 -0.97 -1.4 -0.76 0.13 ]


In [159]:
print(motor)

Mot: [ 0.87 -0.13 -0.27 -0.4 -0.97 -1.4 -0.76 0.13 ]


In [58]:
# jac(init_dll)